# Using Foursquare to predict land prices in São Paulo

_Capstone project for the IBM Professional Data Science Certification program_

In this project we use Foursquare venue locations associated with publicly available land value data to create a model to predict land prices using Regression algorithms.

## Introduction

It is known that close-by ammenities can have an impact on land prices. This project intends to cross information already known from land prices in the city of São Paulo, Brazil, with Foursquare venue locations, in order to create a model capable to predict land prices for a given location in the city.

The main beneficiaries from this report would be real estate investors and agents, followed by anyone searching for a good place to live in this gigantic city. Also, it adds value to the Data Science community, by exposing the results of the conducted research to evaluate if the use of Foursquare can benefit other land price prediction models.

## Data

To perform this research, two main datasets will be used:

- Foursquare venue locations (extracted from the Foursquare API);
- Public data from São Paulo's city hall related with land value for each city address;